In [4]:
import os,shutil
base_dir = 'C:\\Users\\krishan\\GPU-projects\\DogsVsCats'

In [5]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [6]:
from keras.applications import VGG16
conv_base = VGG16(include_top=False,#Dense layer of VGG16 shall be discarded,
                 weights = 'imagenet',
                 input_shape=(150,150,3))
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [9]:
batch_size =20
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator(rescale=1./255)

def extract_features(directory, samplesize):
    features = np.zeros(shape=(samplesize,4,4,512))
    labels = np.zeros((samplesize))
    print(directory)
    generator = datagen.flow_from_directory(directory,
                                            target_size=(150,150),
                                            class_mode='binary',
                                            batch_size = batch_size)

    i=0
    for image_batch,label_batch in generator:
        features_batch = conv_base.predict(image_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = label_batch
        i+=1
        if i*batch_size>=samplesize :
            break
    return features,labels
        
    

In [21]:
train_features,train_labels = extract_features(train_dir,2000)
validation_features,validation_labels = extract_features(val_dir,1000)
test_features,test_labels = extract_features(test_dir,1000)



C:\Users\krishan\GPU-projects\DogsVsCats\train
Found 2000 images belonging to 2 classes.
C:\Users\krishan\GPU-projects\DogsVsCats\validation
Found 1000 images belonging to 2 classes.
C:\Users\krishan\GPU-projects\DogsVsCats\test
Found 1000 images belonging to 2 classes.


In [25]:
train_features = train_features.reshape((len(train_features),4*4*512))
validation_features = validation_features.reshape((len(validation_features),4*4*512))
test_features = test_features.reshape((len(test_features),4*4*512))

train_features.shape

(2000, 8192)

In [26]:
from keras import layers
from keras import Sequential

model = Sequential()
model.add(layers.Dense(512,activation='relu', input_shape=(4*4*512,)))
model.add(layers.Dense(1,activation='sigmoid'))

In [27]:
from keras import losses
from keras import optimizers
model.compile(optimizer=optimizers.adam(lr=1e-4),
             loss='binary_crossentropy',
             metrics = ['acc'])

In [ ]:
history = model.fit(train_features,train_labels,
                   epochs = 100,
                   validation_data=(validation_features,validation_labels),
                   batch_size = 32,
                   )

Train on 2000 samples, validate on 1000 samples
Epoch 1/100
2000/2000 [==============================] - 2s 783us/step - loss: 0.3962 - acc: 0.8145 - val_loss: 0.2684 - val_acc: 0.8890
Epoch 2/100
2000/2000 [==============================] - 1s 345us/step - loss: 0.2143 - acc: 0.9180 - val_loss: 0.2395 - val_acc: 0.8990
Epoch 3/100
2000/2000 [==============================] - 1s 333us/step - loss: 0.1583 - acc: 0.9410 - val_loss: 0.2373 - val_acc: 0.9010
Epoch 4/100
2000/2000 [==============================] - 1s 326us/step - loss: 0.1200 - acc: 0.9615 - val_loss: 0.2661 - val_acc: 0.8850
Epoch 5/100
2000/2000 [==============================] - 1s 334us/step - loss: 0.0952 - acc: 0.9745 - val_loss: 0.2442 - val_acc: 0.9020
Epoch 6/100
2000/2000 [==============================] - 1s 331us/step - loss: 0.0872 - acc: 0.9735 - val_loss: 0.2911 - val_acc: 0.8810
Epoch 7/100
2000/2000 [==============================] - 1s 333us/step - loss: 0.0655 - acc: 0.9845 - val_loss: 0.2422 - val_acc: 

In [ ]:
import matplotlib.pyplot as pp
def plothistory(history):
    pp.figure(figsize=(10,4))
    pp.subplot(1,2,1)
    pp.plot(history['loss'])
    pp.plot(history['val_loss'],'r')
    pp.legend(['loss','val_loss'])
    pp.title('loss')
    pp.subplot(1,2,2)
    pp.plot(history['acc'])
    pp.plot(history['val_acc'],'r')
    pp.legend(['acc','val_acc'])
    pp.title('accuracy')
    pp.show()
plothistory(history.history)

In [ ]:
model.save('cat_anddog_small_2.h5')